In [75]:
import sys
import os

import gcsfs
import numpy as np
import pandas as pd
import wandb
from catboost import CatBoostClassifier, Pool
from numpy.testing import assert_almost_equal
from pandas._testing.asserters import assert_almost_equal
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
from tqdm.auto import tqdm

sys.path.append("..")

In [76]:
exchange = "cboe" # "cboe"
strategy = "supervised" 
max_i = 50 if exchange == "ise" else 38 # number of partial files

In [77]:
# connect to weights and biases
run = wandb.init(project="thesis", job_type="dataset-creation", entity="fbv")
dataset = wandb.Artifact(name=f"{exchange}_{strategy}_raw", type="preprocessed_data")


In [78]:
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

fs = gcsfs.GCSFileSystem(project="thesis")

In [79]:

files = [f"gs://thesis-bucket-option-trade-classification/data/preprocessed/{'matched' if strategy == 'supervised' else 'unmatched'}_{exchange}_quotes_min_mem_usage_extended_part_{i:04d}.parquet" for i in range(0, max_i)]

# asks = [f"ASK_{i}" for i in range(1, 17)]
# bids = [f"BID_{i}" for i in range(1, 17)]

columns = [
    "QUOTE_DATETIME",
    "ROOT",
    "EXPIRATION",
    "STRK_PRC",
    "OPTION_TYPE",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "BEST_BID",
    "BEST_ASK",
    "ask_ex",
    "bid_ex",
    "bid_size_ex",
    "ask_size_ex",
    "price_all_lead",
    "price_all_lag",
    "optionid",
    "day_vol",
    "price_ex_lead",
    "price_ex_lag",
    "issue_type",
    "myn",
    # *asks,
    # *bids,
    "buy_sell",
]

dfs = [pd.read_parquet(gc_file, columns=columns) for gc_file in tqdm(files)]
df = pd.concat(dfs)

del dfs

100%|██████████| 38/38 [01:42<00:00,  2.71s/it]


In [80]:
df.memory_usage(deep=True).sum()


5529034042

In [81]:
df.head().T


,0,1,2,3,4
QUOTE_DATETIME,2011-02-08 15:44:23,2011-02-25 12:05:06,2011-01-24 10:20:49,2011-01-24 10:20:49,2011-01-24 09:50:25
ROOT,A,A,A,A,A
EXPIRATION,2011-03-19 00:00:00,2011-03-19 00:00:00,2011-03-19 00:00:00,2011-03-19 00:00:00,2011-03-19 00:00:00
STRK_PRC,36.0,37.0,38.0,40.0,42.0
OPTION_TYPE,C,C,P,P,C
TRADE_SIZE,1,7,10,10,1
TRADE_PRICE,8.22,5.27,0.62,1.07,2.08
BEST_BID,7.95,5.1,0.59,1.06,2.08
BEST_ASK,8.7,5.4,0.62,1.11,2.16
ask_ex,8.7,5.4,0.63,1.11,2.16


In [82]:
len(df)

37155412

In [83]:
# check against some stats from sub panel A.1 in Grauer et al

# trade size
stats_trade_size = df["TRADE_SIZE"].agg(["mean", "median", "std"])

# time to maturity
stats_time_to_maturity = (df["EXPIRATION"] - df["QUOTE_DATETIME"]).dt.days
stats_time_to_maturity = stats_time_to_maturity.agg(["mean", "median", "std"])

# no of observations
stats_n = len(df)

# trade_size = quote size; TRADE_SIZE
stats_trades_with_quote_size_bid = df["bid_size_ex"].eq(df["TRADE_SIZE"])
stats_trades_with_quote_size_ask = df["ask_size_ex"].eq(df["TRADE_SIZE"])

# ask or bid 
stats_trade_with_quote_size = (
    stats_trades_with_quote_size_bid | stats_trades_with_quote_size_ask
).sum() / stats_n


# no of buys
stats_buy_trades = df["buy_sell"].ge(0).sum() / stats_n


if exchange == "ise" and strategy == "supervised": 
    assert stats_n == 49203747
    assert_almost_equal(stats_trade_with_quote_size, 0.2281, atol=0.01)
    assert_almost_equal(stats_trade_size.values.tolist(), [13.62, 4.0, 77.75], atol=0.1)
    assert_almost_equal(stats_buy_trades, 0.4746, atol=0.01)

if exchange == "cboe" and strategy == "supervised": 
    assert stats_n == 37155412
    assert_almost_equal(stats_trade_with_quote_size, 0.1397, atol=0.01)
    assert_almost_equal(stats_trade_size.values.tolist(), [18.14,5.0, 223.24], atol=0.1)
    assert_almost_equal(stats_buy_trades, 0.4500, atol=0.01)

In [84]:
# 
df["QUOTE_DATETIME"].sort_values().iloc[int(len(df)*0.8)]

Timestamp('2016-10-13 09:30:58')

In [85]:
df.head()

,QUOTE_DATETIME,ROOT,EXPIRATION,STRK_PRC,OPTION_TYPE,TRADE_SIZE,TRADE_PRICE,BEST_BID,BEST_ASK,ask_ex,...,ask_size_ex,price_all_lead,price_all_lag,optionid,day_vol,price_ex_lead,price_ex_lag,issue_type,myn,buy_sell
0,2011-02-08 15:44:23,A,2011-03-19,36.0,C,1,8.22,7.95,8.70,8.70,...,47.0,8.00,NaN,51176928.0,1.0,8.00,NaN,0,1.229583,-1
1,2011-02-25 12:05:06,A,2011-03-19,37.0,C,7,5.27,5.10,5.40,5.40,...,35.0,NaN,NaN,51176928.0,7.0,NaN,NaN,0,1.135000,1
2,2011-01-24 10:20:49,A,2011-03-19,38.0,P,10,0.62,0.59,0.62,0.63,...,23.0,0.58,NaN,50197824.0,10.0,0.76,NaN,0,0.901328,1
3,2011-01-24 10:20:49,A,2011-03-19,40.0,P,10,1.07,1.06,1.11,1.11,...,33.0,0.99,NaN,50197824.0,10.0,0.85,NaN,0,0.948767,-1
4,2011-01-24 09:50:25,A,2011-03-19,42.0,C,1,2.08,2.08,2.16,2.16,...,21.0,2.30,NaN,50197816.0,1.0,1.94,NaN,0,1.006190,-1


## train-test-split ⚗️

In [86]:
df.sort_values(by="QUOTE_DATETIME", inplace=True)

In [87]:
# indices 

if exchange == "ise" and strategy == "supervised":
    train_range = df.QUOTE_DATETIME.between("2005-05-02 00:00:01", "2013-10-24 23:59:00")
    val_range = df.QUOTE_DATETIME.between("2013-10-25 00:00:01", "2015-11-05 23:59:00")
    test_range = df.QUOTE_DATETIME.between("2015-11-06 00:00:01", "2017-05-31 23:59:00")
if exchange == "cboe" and strategy ==  "supervised":
    train_range = df.QUOTE_DATETIME.between("2011-01-01 00:00:01", "2015-06-15 23:59:00")
    val_range = df.QUOTE_DATETIME.between("2015-06-16 00:00:01", "2016-10-12 23:59:00")
    test_range = df.QUOTE_DATETIME.between("2016-10-13 00:00:01", "2017-10-31 23:59:00")  
    
train = df[train_range]

len_train = len(train)
print(f"train ratio: {len_train / len(df)}")


train ratio: 0.6000126711015881


In [88]:
val = df[val_range]
len_val = len(val)
print(f"val ratio: {len_val / len(df)}")


val ratio: 0.19996893050196832


In [89]:
test = df[test_range]
len_test = len(test)
print(f"test ratio: {len_test / len(df)}")


test ratio: 0.20001839839644356


In [90]:
# check if total is sum of its parts
assert len_train + len_val + len_test == len(df)


In [91]:
train


,QUOTE_DATETIME,ROOT,EXPIRATION,STRK_PRC,OPTION_TYPE,TRADE_SIZE,TRADE_PRICE,BEST_BID,BEST_ASK,ask_ex,...,ask_size_ex,price_all_lead,price_all_lag,optionid,day_vol,price_ex_lead,price_ex_lag,issue_type,myn,buy_sell
799115,2011-01-03 09:30:04,RWM,2011-04-16,34.0,C,3,1.10,0.00,0.00,NaN,...,NaN,1.25,1.40,37533748.0,3.0,1.25,1.40,%,NaN,1
808408,2011-01-03 09:30:04,CDE,2011-02-19,26.0,C,111,2.80,0.00,0.00,NaN,...,NaN,3.00,2.75,47012424.0,117.0,2.69,3.10,0,NaN,1
811820,2011-01-03 09:30:04,UNG,2011-01-22,13.0,P,3,6.85,6.75,6.85,NaN,...,NaN,6.79,7.00,34267312.0,3.0,6.92,7.19,%,2.091714,1
813492,2011-01-03 09:30:04,FXC,2011-01-22,100.0,C,1,0.95,0.95,1.10,NaN,...,NaN,1.05,0.80,49744108.0,1.0,0.65,0.75,%,1.003550,-1
812175,2011-01-03 09:30:04,CDE,2011-01-22,26.0,C,2,2.15,0.00,0.00,NaN,...,NaN,1.90,1.95,37531324.0,2.0,0.90,1.95,0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22678360,2015-06-15 16:14:42,SPX,2015-12-19,1350.0,P,10,3.20,2.75,3.30,3.30,...,1.0,3.30,3.30,100269240.0,41.0,3.30,3.30,A,0.647662,-1
22678390,2015-06-15 16:14:50,SPX,2015-12-19,1350.0,P,10,3.20,2.75,3.30,3.30,...,3.0,3.30,3.30,100269240.0,41.0,3.30,3.30,A,0.647662,-1
22678391,2015-06-15 16:14:53,VXX,2015-06-26,21.5,C,2,0.31,0.30,0.32,0.33,...,104.0,0.27,0.30,107693848.0,149.0,0.27,0.30,%,0.892326,-1
22678392,2015-06-15 16:14:57,SPX,2015-12-19,1350.0,P,10,3.20,2.75,3.20,3.20,...,10.0,3.30,3.30,100269240.0,41.0,3.30,3.30,A,0.647662,-1


In [92]:
val


,QUOTE_DATETIME,ROOT,EXPIRATION,STRK_PRC,OPTION_TYPE,TRADE_SIZE,TRADE_PRICE,BEST_BID,BEST_ASK,ask_ex,...,ask_size_ex,price_all_lead,price_all_lag,optionid,day_vol,price_ex_lead,price_ex_lag,issue_type,myn,buy_sell
22335817,2015-06-16 09:30:02,DEM,2015-09-18,42.0,P,1,1.600000,1.100000,1.600000,1.600000,...,34.0,0.80,0.750000,106541376.0,1.0,1.20,2.65,%,0.957374,1
22676439,2015-06-16 09:30:02,SLV,2015-10-16,14.5,P,1,0.530000,0.530000,0.570000,NaN,...,NaN,0.52,0.500000,106339344.0,1.0,0.47,0.50,%,0.949885,-1
22672774,2015-06-16 09:30:02,RSX,2015-08-21,14.0,P,3,0.050000,0.050000,0.150000,0.150000,...,13.0,0.14,0.100000,106333840.0,3.0,0.06,0.10,%,0.750469,-1
22609680,2015-06-16 09:30:02,XBI,2015-06-19,190.0,C,2,48.900002,48.900002,48.700001,48.700001,...,2.0,52.16,48.200001,105770880.0,3.0,52.16,46.00,%,1.264000,-1
22678250,2015-06-16 09:30:02,FB,2016-01-15,70.0,C,1,13.500000,13.300000,14.350000,NaN,...,NaN,13.77,13.600000,102041552.0,1.0,14.30,13.62,0,1.153929,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29928978,2016-10-12 16:14:53,SPX,2017-01-20,1520.0,P,30,2.050000,1.850000,2.350000,2.350000,...,10.0,2.75,2.000000,113253400.0,600.0,2.75,2.00,A,0.710368,-1
29939247,2016-10-12 16:14:54,VXX,2016-11-11,33.5,P,5,2.230000,2.180000,2.260000,2.260000,...,652.0,1.80,2.210000,113358320.0,8.0,1.82,2.21,%,0.965418,1
29939245,2016-10-12 16:14:54,VXX,2016-11-11,32.5,P,5,1.690000,1.660000,1.720000,1.720000,...,82.0,1.40,1.670000,113358320.0,8.0,1.40,1.67,%,0.936599,-1
29933986,2016-10-12 16:14:54,SPY,2016-12-30,190.0,P,2,1.300000,1.290000,1.310000,1.330000,...,579.0,1.59,1.250000,110333192.0,2.0,1.60,1.45,%,0.889971,-1


In [93]:
test


,QUOTE_DATETIME,ROOT,EXPIRATION,STRK_PRC,OPTION_TYPE,TRADE_SIZE,TRADE_PRICE,BEST_BID,BEST_ASK,ask_ex,...,ask_size_ex,price_all_lead,price_all_lag,optionid,day_vol,price_ex_lead,price_ex_lag,issue_type,myn,buy_sell
29930521,2016-10-13 09:30:02,SLV,2016-10-28,19.0,C,1,0.020000,0.020000,0.030000,0.030000,...,864.0,0.01,0.01,113083320.0,2.0,NaN,0.010000,%,0.873421,1
29894717,2016-10-13 09:30:02,CAH,2016-10-21,72.5,P,10,0.270000,0.050000,0.300000,0.400000,...,10.0,0.30,0.10,112820032.0,11.0,0.30,0.290000,0,0.962496,1
29909660,2016-10-13 09:30:02,ETP,2016-12-16,43.0,C,1,0.200000,0.200000,0.500000,0.500000,...,25.0,0.17,0.26,111565888.0,3.0,0.17,0.260000,0,0.837907,1
29939322,2016-10-13 09:30:02,QQQ,2016-10-28,120.5,C,1,0.120000,0.120000,0.150000,0.160000,...,535.0,0.13,0.11,113081056.0,12.0,0.13,0.190000,%,0.968091,-1
29883818,2016-10-13 09:30:02,CDE,2016-12-16,10.0,C,5,1.850000,1.600000,1.850000,1.850000,...,47.0,1.60,1.90,111450416.0,5.0,1.86,1.900000,0,1.097000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37155409,2017-10-31 16:14:40,RUT,2017-12-15,1420.0,C,1,90.699997,89.599998,91.800003,91.800003,...,10.0,NaN,91.00,114648504.0,3.0,NaN,77.730003,A,1.058468,1
37155410,2017-10-31 16:14:41,VXX,2017-12-01,43.5,P,122,10.600000,10.400000,10.550000,10.550000,...,20.0,NaN,8.00,117640752.0,122.0,NaN,8.050000,%,1.283186,1
37104659,2017-10-31 16:14:48,VIX,2018-03-21,37.5,C,450,0.400000,0.350000,0.400000,0.400000,...,450.0,NaN,0.45,117441288.0,450.0,NaN,0.450000,A,0.270133,1
37108365,2017-10-31 16:14:57,VIX,2018-01-17,35.0,C,500,0.250000,0.250000,0.300000,0.300000,...,22753.0,NaN,0.35,116597256.0,1000.0,NaN,0.350000,A,0.290857,1


In [94]:
output_path = f"gs://thesis-bucket-option-trade-classification/data/preprocessed/{exchange}_{strategy}_train.parquet"
train.to_parquet(output_path)
dataset.add_reference(output_path, name="train_set")


/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


[<ManifestEntry ref: gs://thesis-bucket-option-trade-classification/data/preprocessed/cboe_supervised_train.parquet/train_set>]

In [95]:
output_path = f"gs://thesis-bucket-option-trade-classification/data/preprocessed/{exchange}_{strategy}_val.parquet"
val.to_parquet(output_path)
dataset.add_reference(output_path, name="val_set")


[<ManifestEntry ref: gs://thesis-bucket-option-trade-classification/data/preprocessed/cboe_supervised_val.parquet/val_set>]

In [96]:
output_path = f"gs://thesis-bucket-option-trade-classification/data/preprocessed/{exchange}_{strategy}_test.parquet"
test.to_parquet(output_path)
dataset.add_reference(output_path, name="test_set")


[<ManifestEntry ref: gs://thesis-bucket-option-trade-classification/data/preprocessed/cboe_supervised_test.parquet/test_set>]

In [97]:
# Log the artifact to save it as an output of this run
run.log_artifact(dataset)

wandb.finish()


## Relevant length of dataset⏲️

In [2]:
train = pd.read_parquet(
    "gs://thesis-bucket-option-trade-classification/data/ise_log_standardized/train_set.parquet",
    engine="fastparquet"
)
val = pd.read_parquet(
    "gs://thesis-bucket-option-trade-classification/data/ise_log_standardized/val_set.parquet",
    engine="fastparquet"
)

/home/kit/stud/uloak/.local/lib/python3.8/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
val = pd.read_parquet(
    "gs://thesis-bucket-option-trade-classification/data/ise_log_standardized/val_set_20.parquet",
    engine="fastparquet"
)

In [4]:
y_train = train["buy_sell"]
X_train = train.drop(columns=["buy_sell"])

In [5]:
y_val = val["buy_sell"]
X_val = val.drop(columns=["buy_sell"])

In [6]:
X_train.head()

,TRADE_PRICE,bid_ask_size_ratio_ex,rel_bid_size_ex,rel_ask_size_ex,depth_ex,prox_ex,prox_best,spread_ex,spread_best,bid_ask_ratio_ex,...,date_month_cos,date_day_sin,date_day_cos,date_weekday_sin,date_weekday_cos,date_time_sin,date_time_cos,bin_option_type,bin_issue_type,bin_root
0,-0.075434,-0.056979,-0.080504,0.050436,0.077184,0.596202,0.545705,-0.022887,-0.000591,0.263251,...,-0.866025,0.394356,0.918958,0.0,1.0,0.608646,-0.793442,0,1,8337
1,0.435948,-0.078490,-0.105558,-0.108601,0.005799,0.027073,2.111057,-0.487660,-0.000591,-3.523268,...,-0.866025,0.394356,0.918958,0.0,1.0,0.608588,-0.793486,0,1,5899
2,1.419895,-0.075206,-0.050857,-0.055588,0.005799,-0.352349,-0.323935,0.209498,0.000184,0.551698,...,-0.866025,0.394356,0.918958,0.0,1.0,0.608588,-0.793486,0,0,5864
3,-1.081623,-0.078490,-0.105558,-0.071615,-0.049509,0.710028,0.650062,0.093306,-0.000203,-3.523268,...,-0.866025,0.394356,0.918958,0.0,1.0,0.608588,-0.793486,0,1,4943
4,-1.037590,-0.050867,-0.104091,-0.096643,1.907473,-1.111186,-2.063210,-0.022887,-0.001365,-1.198212,...,-0.866025,0.394356,0.918958,0.0,1.0,0.608588,-0.793486,0,1,4458


In [7]:
results_p = []
percentages = np.linspace(0.1, 1, 10)


# go back from last element
for p in tqdm(percentages):
	# keep ordering of data
	length = int(len(y_train) * p)
	timestamp = np.linspace(0, 1, length)
	# keep weight fixed
	for strategy in ["uniform", "exponential"]:

		if strategy == "uniform":
			weight = np.ones(length)
		else:
			weight = np.geomspace(0.001, 1, num=len(y_train))[-length:]

		train_pool = Pool(
			data=X_train.iloc[-length:],
			label=y_train.iloc[-length:],
			# cat_features=cat_features,
			weight=weight,
			timestamp=np.linspace(0, 1, length),
		)
		val_pool = Pool(data=X_val.iloc[-length:], label=y_val.iloc[-length:])

		kwargs_cat = {
			# "iterations": 1000,
			# "grow_policy": "symmetric",
			# "border_count": 254,
			"logging_level": "Silent",
			"task_type": "GPU",
			"random_seed": 42,
			"eval_metric": "Accuracy",
			# "early_stopping_rounds": 100,
		}

		clf = CatBoostClassifier(**kwargs_cat)
		clf.fit(
			train_pool,
			eval_set=val_pool,
		)

		train_acc = clf.score(train_pool)
		val_acc = clf.score(val_pool)

		res = {
			"start": -length,
			"end": -1,
			"train_acc": train_acc,
			"val_acc": val_acc,
			"strategy": strategy,
		}
		print(res)
		results_p.append(res)


  0%|          | 0/10 [00:00<?, ?it/s]

{'start': -2951032, 'end': -1, 'train_acc': 0.8503154150819103, 'val_acc': 0.7097832215984103, 'strategy': 'uniform'}
{'start': -2951032, 'end': -1, 'train_acc': 0.8502127391366817, 'val_acc': 0.7102562764483746, 'strategy': 'exponential'}
{'start': -5902064, 'end': -1, 'train_acc': 0.8560598461826235, 'val_acc': 0.7301994692026382, 'strategy': 'uniform'}
{'start': -5902064, 'end': -1, 'train_acc': 0.8558826200461398, 'val_acc': 0.7314344608936806, 'strategy': 'exponential'}
{'start': -8853096, 'end': -1, 'train_acc': 0.8605411033609034, 'val_acc': 0.7502536965599379, 'strategy': 'uniform'}
{'start': -8853096, 'end': -1, 'train_acc': 0.8597372037985356, 'val_acc': 0.7516163836922134, 'strategy': 'exponential'}
{'start': -11804128, 'end': -1, 'train_acc': 0.8644420833118719, 'val_acc': 0.7552953151954804, 'strategy': 'uniform'}
{'start': -11804128, 'end': -1, 'train_acc': 0.8623050343066425, 'val_acc': 0.7575980352021201, 'strategy': 'exponential'}
{'start': -14755160, 'end': -1, 'train

In [8]:
results_df = pd.DataFrame(results_p)

In [9]:
results_df

,start,end,train_acc,val_acc,strategy
0,-2951032,-1,0.850315,0.709783,uniform
1,-2951032,-1,0.850213,0.710256,exponential
2,-5902064,-1,0.856060,0.730199,uniform
3,-5902064,-1,0.855883,0.731434,exponential
4,-8853096,-1,0.860541,0.750254,uniform
5,-8853096,-1,0.859737,0.751616,exponential
6,-11804128,-1,0.864442,0.755295,uniform
7,-11804128,-1,0.862305,0.757598,exponential
8,-14755160,-1,0.868171,0.753912,uniform
9,-14755160,-1,0.864821,0.757307,exponential


In [10]:
results_df.to_csv("learning_curves_gbm_default_params.csv")

## Time consistency

Check if features maintain their predictive power over time, buy training on the first $10~\%$ of the training set and predicting the last $10~\%$ feature by feature. Remove features or further investigate features where accuracy is just above or below $0.5$. Technique found in this [notebook](https://www.kaggle.com/code/cdeotte/xgb-fraud-with-magic-0-9600/notebook).

In [4]:
data = pd.read_parquet(
    "gs://thesis-bucket-option-trade-classification/data/classical_size_features_log_normalized/train_set_extended_60.parquet",
    engine="fastparquet"
)

In [5]:
# try to predict last 10 % in training set using first 10 % of features. Accuracy should be above 50 %.
label = data["buy_sell"]
data.drop(columns=["buy_sell"], inplace=True)

In [6]:
y_train = label.iloc[0:len(label)//10]
y_test = label.iloc[-len(data)//10:]

X_train = data.iloc[0:len(data)//10,:]
X_test = data.iloc[-len(data)//10:,:]

del label, data

In [7]:
y_train.shape

(2951032,)

In [8]:
params = {
        "od_type": "Iter",
        "logging_level": "Silent",
        "loss_function": "Logloss",
        "task_type": "GPU",
        "cat_features": None,
        "random_seed": 42,
        "eval_metric":"Accuracy",
        "iterations":1000,
        "early_stopping_rounds":100,
}

In [13]:
columns = X_train.columns

In [21]:
results = []
for col in tqdm(columns):
  model = CatBoostClassifier(**params)
  model.fit(X_train[[col]], y_train, eval_set=(X_test[[col]],y_test))
  acc = model.score(X_test[[col]], y_test)
  results.append([col,acc])

  0%|          | 0/24 [00:00<?, ?it/s]

In [24]:
results_df = pd.DataFrame(results, columns=["feature", "accuracy"])
results_df.sort_values(by="accuracy")

,feature,accuracy
10,chg_ex_lag,0.518701
21,TRADE_SIZE,0.522728
9,chg_ex_lead,0.527728
12,chg_all_lag,0.547701
8,bid_ask_ratio_ex,0.548045
3,rel_ask_size_ex,0.550789
23,ask_size_ex,0.552150
19,price_ex_lag,0.552589
11,chg_all_lead,0.554612
0,TRADE_PRICE,0.554639


Few features are actually consistent over time and are more informative than a random guess. These include entire includes features related to the proximity of the quote and relative bid size.

Some features like `chg_ex_lead` are hard to exclude. Better weight observations, as suggested in `3.0c-feature-engineering.ipynb`.